# This set of functions can be used to load the ARXIV data file and send it to the AWS queue

sciml_data_arxiv.p  is  the set of original arxiv rss items.  a subset of these were use for training.  the url is https://1drv.ms/u/s!AkRG9Zk_IOUagrFxuO3yxvluvwQ1uA This is a OneDrive link so you will need to go there on your browser and go to the "download" tab at the top and download the file.   perhaps in "/tmp"

sciml_data_arxiv_new_9_28_15.p  "newly" arrived arxiv items (not in the above) it is at https://1drv.ms/u/s!AkRG9Zk_IOUagrFvwrNNB8Jgc1ARcA   you can also use this one.  it does not contain anything  that was used for training.



In [2]:
import pickle
import json
import boto3

The following function just loads the pickled data file  and extracts the titles, sitenames (arxiv designations) and abstracts

In [3]:
def load_docs(path, name):
    filename = path+name+".p"
    fileobj = open(filename, "rb")
    z = fileobj.read()
    lst = pickle.loads(str(z))
    titles = []
    sitenames = []
    abstracts = []
    for i in range(0, len(lst)):
        titles.extend([lst[i][0]])
        sitenames.extend([lst[i][1]])
        abstracts.extend([lst[i][2]])
        
    print "done loading "+filename
    return abstracts, sitenames, titles

this call loads three arrays: abstracts: the paper abstract,  sites: the arxiv designation, and the title

In [4]:
abstracts, sites, titles = load_docs("c:/Users/dennis/Desktop/tutorial/", "sciml_data_arxiv")

done loading c:/Users/dennis/Desktop/tutorial/sciml_data_arxiv.p


let's take a peek at one of them.  how about number 1301

In [5]:
print abstracts[303]
print sites[303]
print titles[303]

The selection process of proposals is a crucial component of scientific progress and innovations. Limited resources must be allocated in the most effective way to maximise advancements and the production of new knowledge, especially as it is becoming increasingly clear that technological and scientific innovation and creativity is an instrument of economic policy and social development. The traditional approach based on merit evaluation by experts has been the preferred method, but there is an issue regarding to what extent such a method can also be an instrument of effective policy. This paper discuss some of the basic processes involved in the evaluation and selection of proposals, indicating some criterion for an optimal solution.
physics.soc-ph
The choice: evaluating and selecting scientific proposals [physics.soc-ph]


In [5]:
sqs = boto3.resource('sqs',
    region_name='us-west-2', 
 )


# now let's grab the queue
The queue "bookque" was previously created from the aws portal

In [6]:
queuename = 'bookque'
queue = sqs.get_queue_by_name(QueueName=queuename)
print(queue.url)
print(queue.attributes.get('DelaySeconds'))


https://us-west-2.queue.amazonaws.com/066301190734/bookque
0


In [7]:
import time

# send data to the queue
this loop grabs 100 papers (1350 through 1449) and pushes them to the queue.  These 100 were chosen as a random sample.  nothing special about them. 

In [ ]:
t0 = time.time()
for i in range(950,1450):
    abstract = abstracts[i]
    source = sites[i]
    title = titles[i]
    #print abstract
    print title
    queue.send_message(MessageBody='boto3', MessageAttributes ={
            'Title':{ 'StringValue': title,
                      'DataType': 'String'}, 
            'Source':{ 'StringValue': source,
                      'DataType': 'String'}, 
            'Abstract':{ 'StringValue': abstract,
                      'DataType': 'String'} 
        })
t1 = time.time()

Characterization of metrics induced by hierarchical posets [cs.IT]
An algorithm for online tensor prediction [stat.ML]
A note on the clique number of complete $k$-partite graphs [cs.DM]
Supervised Collective Classification for Crowdsourcing [cs.SI]
Breaking the News: First Impressions Matter on Online News [cs.CY]
Work Capacity of Freelance Markets: Fundamental Limits and Decentralized   Schemes [cs.MA]
An Analysis of Device-Free and Device-Based WiFi-Localization Systems [cs.NI]
Practical Run-time Checking via Unobtrusive Property Caching [cs.PL]
Adaptive Normalized Risk-Averting Training For Deep Neural Networks [cs.LG]
Improved Core Genes Prediction for Constructing well-supported   Phylogenetic Trees in large sets of Plant Species [q-bio.GN]
Numerical relativity simulations of neutron star merger remnants using   conservative mesh refinement [gr-qc]
Approximate Joint Diagonalization and Geometric Mean of Symmetric   Positive Definite Matrices [math.DG]
Queuing models for abstractin

In [15]:
print 'elapsed time ='+str(t1-t0)

elapsed time =3.36903595924


# what was in the dynamoDB table after the process ended?
To see the output that appeared in the dynodb table look at BookTable-dynammodb-output.csv

Note: it took 3 seconds to fill the queue and if we look at the csv file and sort by timestame we see it took 26 seconds to do all the processing using 8 predictors.  